# Resume Experiment Analysis

How much harder is it to get a job in the United States if you are Black than if you are White? Or, expressed differently, what is the *effect* of race on the difficulty of getting a job in the US?

In this exercise, we will be analyzing data from a real world experiment designed to help answer this question. Namely, we will be analyzing data from a randomized experiment in which 4,870 ficticious resumes were sent out to employers in response to job adverts in Boston and Chicago in 2001. The resumes differ in various attributes including the names of the applicants, and different resumes were randomly allocated to job openings. 

The "experiment" part of the experiment is that resumes were randomly assigned Black- or White-sounding names, and then watched to see whether employers called the "applicants" with Black-sounding names at the same rate as the applicants with the White-sounding names.

(Which names constituted "Black-sounding names" and "White-sounding names" was determined by analyzing names on Massachusetts birth certificates to determine which names were most associated with Black and White children, and then surveys were used to validate that the names were perceived as being associated with individuals of one racial category or the other. Also, please note I subscribe to the logic of [Kwame Anthony Appiah](https://www.theatlantic.com/ideas/archive/2020/06/time-to-capitalize-blackand-white/613159/) and chose to capitalize both the B in Black and the W in White). 

You can get access to original article [here](https://www.aeaweb.org/articles?id=10.1257/0002828042002561). 

**Note to Duke students:** if you are on the Duke campus network, you'll be able to access almost any academic journal articles directly; if you are off campus and want access, you can just go to the [Duke Library](https://library.duke.edu/) website and search for the article title. Once you find it, you'll be asked to log in, after which you'll have full access to the article. You will also find this pattern holds true at nearly any major University in the US.



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/ids720_specific/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_resume_experiment.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex2_pvalue_computerskills",
    "ex2_pvalue_female",
    "ex2_pvalue_yearsexp",
    "ex3_pvalue_education",
    "ex4_validity",
    "ex5_pvalue",
    "ex5_white_advantage_percent",
    "ex5_white_advantage_percentage_points",
    "ex6_black_pvalue",
    "ex8_black_college",
    "ex8_black_nocollege",
    "ex8_college_heterogeneity",
    "ex9_gender_and_discrimination",
    "ex10_experiment_v_us",
}
```


### Submission Limits

Please remember that you are **only allowed THREE submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Checking for Balance

The first step in analyzing any experiment is to check whether you have *balance* across your treatment arms—that is to say, do the people who were randomly assigned to the treatment group look like the people who were randomly assigned to the control group. Or in this case, do the resumes that ended up with Black-sounding names look like the resumes with White-sounding names. 

Checking for balance is critical for two reasons. First, it's always possible that random assignment will create profoundly different groups—the *Large of Large Numbers* is only a "law" in the limit. So we want to make sure we have reasonably similar groups from the outset. And second, it's also always possible that the randomization wasn't actually implemented correctly—you would be amazed at the number of ways that "random assignment" can go wrong! So if you ever do find you're getting unbalanced data, you should worry not only about whether the groups have baseline differences, but also whether the "random assignment" was actually random!

### Exercise 1

Download the data set from this experiment (`resume_experiment.dta`) from [github](https://github.com/nickeubank/MIDS_Data/tree/master/resume_experiment). To aid the autograder, please load the data directly from a URL.


In [1]:
import pandas as pd
import numpy as np

url = "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/resume_experiment/resume_experiment.dta"
df = pd.read_stata(url)
results = {}

df.head()

,education,ofjobs,yearsexp,computerskills,call,female,black
0,4,2,6,1,0.0,1.0,0.0
1,3,3,6,1,0.0,1.0,0.0
2,4,1,6,1,0.0,1.0,1.0
3,3,4,6,1,0.0,1.0,1.0
4,3,3,22,1,0.0,1.0,0.0



### Exercise 2

- `black` is the treatment variable in the data set (whether the resume has a "Black-sounding" name).
- `call` is the dependent variable of interest (did the employer call the fictitious applicant for an interview)

In addition, the data include a number of variables to describe the other features in each fictitious resume, including applicants education level (`education`), years of experience (`yearsexp`), gender (`female`), computer skills (`computerskills`), and number of previous jobs (`ofjobs`). Each resume has a random selection of these attributes, so on average the Black-named fictitious applicant resumes have the same qualifications as the White-named applicant resumes. 

Check for balance in terms of the average values of applicant gender (`female`), computer skills (`computerskills`), and years of experience (`yearsexp`) across the two arms of the experiment (i.e. by `black`). Calculate both the differences in means across treatment arms *and* test for statistical significance of these differences. Does gender, computer skills, and yearsexp look balanced across race groups in terms of both statistical significance and magnitude of difference?

Store the p-values associated with your t-test of these variables in `ex2_pvalue_female`, `ex2_pvalue_computerskills`, and `ex2_pvalue_yearsexp`. **Round your values to 2 decimal places.**


In [2]:
from scipy import stats

# List of variables to check
vars_to_check = ["female", "computerskills", "yearsexp"]

for var in vars_to_check:
    # Split the data into two groups based on 'black' variable
    group_white = df[df["black"] == 0][var]
    group_black = df[df["black"] == 1][var]

    # Calculate difference in means
    diff = group_black.mean() - group_white.mean()

    # Perform t-test
    t_stat, p_val = stats.ttest_ind(group_white, group_black)

    # Store results rounded to two decimal places
    results[f"ex2_pvalue_{var}"] = round(p_val, 2)

    print(f"Variable: {var}, Difference in Means: {diff:.2f}, P-value: {p_val:.2f}")

Variable: female, Difference in Means: 0.01, P-value: 0.38
Variable: computerskills, Difference in Means: 0.02, P-value: 0.03
Variable: yearsexp, Difference in Means: -0.03, P-value: 0.85


### Exercise 3

Do a similar tabulation for education (`education`). Education is a categorical variable coded as follows:

- 0: Education not reported
- 1: High school dropout
- 2: High school graduate
- 3: Some college
- 4: College graduate or higher

Because these are categorical, you shouldn't just calculate and compare means—you should compare share or count of observations with each value (e.g., a chi-squared contingency table). You may also find the `pd.crosstab` function useful.

Does education look balanced across racial groups?

Store the p-value from your chi squared test in results under the key `ex3_pvalue_education`. **Please round to 2 decimal places.**

In [3]:
# Contingency table
contingency_table = pd.crosstab(df["black"], df["education"])

# Chi-squared test
chi2, p_val_edu, dof, expected = stats.chi2_contingency(contingency_table)

# Format and store the p-value
results["ex3_pvalue_education"] = round(p_val_edu, 2)
print(f"Education p-value: {p_val_edu:.2f}")

Education p-value: 0.49


### Exercise 4

What do you make of the overall results on resume characteristics? Why do we care about whether these variables look similar across the race groups? And if they didn't look similar, would that be a threat to internal or external validity? 

Answer in markdown, then also store your answer to the question of whether imbalances are a threat to internal or external validity in `"ex4_validity"` as the string `"internal"` or `"external"`.


In [4]:
results["ex4_validity"] = "internal"

The randomization was generally successful because most variables such as gender, experience, and education show no significant differences between the two groups ($p > 0.05$). Although computer skills show a small statistical imbalance ($p = 0.03$), the actual magnitude of this difference is very minor.

Balance is important because it ensures that race is the only systematic difference between the candidates. If these characteristics were not similar, it would be a threat to internal validity because we could not isolate the true effect of race.

## Estimating Effect of Race

### Exercise 5

The variable of interest in the data set is the variable `call`, which indicates a call back for an interview. Perform a two-sample t-test comparing applicants with black sounding names and white sounding names.

Interpret your results—in both percentage *and* in percentage points, what is the effect of having a Black-sounding name (as opposed to a White-sounding name) on your resume?

Store how much more likely a White applicant is to receive a call back than a Black respondent in percentage and percentage points in `"ex5_white_advantage_percent"`and `"ex5_white_advantage_percentage_points"`. Please scale percentages so 1 is 1% and percentage points so a value of `1` corresponds to 1 percentage point. **Please round these answers to 2 decimal places.**

Store the p-value of the difference in `"ex5_pvalue"` **Please round your p-value to 5 decimal places.**

In [5]:
# Calculate means for 'call' variable by 'black' status
mean_white = df[df["black"] == 0]["call"].mean()
mean_black = df[df["black"] == 1]["call"].mean()

# Percentage points difference
diff_pct_points = (mean_white - mean_black) * 100
results["ex5_white_advantage_percentage_points"] = round(diff_pct_points, 2)

# Percentage difference
white_advantage_pct = ((mean_white - mean_black) / mean_black) * 100
results["ex5_white_advantage_percent"] = round(white_advantage_pct, 2)

# Perform t-test
t_stat_ex5, p_val_ex5 = stats.ttest_ind(
    df[df["black"] == 0]["call"], df[df["black"] == 1]["call"]
)
results["ex5_pvalue"] = round(p_val_ex5, 5)

# Print results
print(f"Mean call rate for White applicants: {mean_white:.4f}")
print(f"Mean call rate for Black applicants: {mean_black:.4f}")

print(f"White advantage (percentage points): {diff_pct_points:.2f}")
print(f"White advantage (percent): {white_advantage_pct:.2f}")
print(f"P-value: {p_val_ex5:.5f}")

Mean call rate for White applicants: 0.0965
Mean call rate for Black applicants: 0.0645
White advantage (percentage points): 3.20
White advantage (percent): 49.68
P-value: 0.00004


Having a Black-sounding name on a resume reduces the callback rate by 3.20 percentage points compared to having a White-sounding name. In relative terms, this means that applicants with Black-sounding names are 33.16 percent less likely to receive a callback than their White-sounding counterparts.

### Exercise 6

Now, use a linear probability model (a linear regression with a 0/1 dependent variable!) to estimate the differential likelihood of being called back by applicant race (i.e. the racial discrimination by employers). Please use [statsmodels](https://www.statsmodels.org/stable/index.html).

Since we have a limited dependent variable, be sure to use [heteroskedastic robust standard errors.](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.get_robustcov_results.html) Personally, I prefer the `HC3` implementation, as it tends to do better with smaller samples than other implementations.

Interpret these results—what is the *effect* of having a Black-sounding name (as opposed to a White-sounding name) on your resume in terms of the likelihood you'll be called back? 

How does this compare to the estimate you got above in exercise 5?

Store the p-value associated with `black` in `"ex6_black_pvalue"`. **Please round your pvalue to 5 decimal places.**

In [6]:
import statsmodels.formula.api as smf

# Define the LPM model
model_ex6 = smf.ols("call ~ black", data=df)

# Fit the model with HC3 robust standard errors
results_ex6 = model_ex6.fit(cov_type="HC3")

# Extract the p-value for the 'black' coefficient
p_val_black = results_ex6.pvalues["black"]
results["ex6_black_pvalue"] = round(p_val_black, 5)

# Print the summary of the regression results
print(results_ex6.summary())

                            OLS Regression Results                            
Dep. Variable:                   call   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     16.92
Date:                Mon, 26 Jan 2026   Prob (F-statistic):           3.96e-05
Time:                        11:54:25   Log-Likelihood:                -562.24
No. Observations:                4870   AIC:                             1128.
Df Residuals:                    4868   BIC:                             1141.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0965      0.006     16.121      0.0

- The coefficient: The coefficient for Black is approximately -0.032. This means that having a Black-sounding name is associated with a 3.2 percentage point decrease in the likelihood of receiving a callback compared with a White-sounding name.

- Comparison to Exercise 5: This estimate matches the difference in means calculated in Exercise 5.

- Statistical significance: With a p-value likely far below 0.05, we reject the null hypothesis that race has no effect on callback rates.

### Exercise 7

Even when doing a randomized experiment, adding control variables to your regression *can* improve the statistical efficiency of your estimates of the treatment effect (the upside is the potential to explain residual variation; the downside is more parameters to be estimated). Adding controls can be particularly useful when randomization left some imbalances in covariates (which you may have seen above). 

Now let's see if we can improve our estimates by adding in other variables as controls. Add in `education`, `yearsexp`, `female`, and `computerskills`—be sure to treat education as a categorical variable!

In [7]:
# Adding controls
formula = "call ~ black + C(education) + yearsexp + female + computerskills"

# Fit the model with HC3 robust standard errors
model_ex7 = smf.ols(formula, data=df).fit(cov_type="HC3")

print(model_ex7.summary())

                            OLS Regression Results                            
Dep. Variable:                   call   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     4.350
Date:                Mon, 26 Jan 2026   Prob (F-statistic):           3.04e-05
Time:                        11:54:30   Log-Likelihood:                -551.02
No. Observations:                4870   AIC:                             1120.
Df Residuals:                    4861   BIC:                             1178.
Df Model:                           8                                         
Covariance Type:                  HC3                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0821      0.04

## Estimating Heterogeneous Effects

### Exercise 8

As you may recall from some past readings (such as this one on the [migraine medication Aimovig](https://ds4humans.com/30_questions/15_answering_exploratory_questions.html#faithful-representations)), our focus on estimating *Average Treatment Effects* runs the risk of papering over variation in how individuals respond. In the case of Aimovig, for example, nearly no patients actually experienced the Average Treatment Effect of the medication; around half of patients experienced no benefit, while the other half experienced a benefit of about twice the average treatment effect.

So far in this analysis we've been focusing on the *average* effect of having a Black-sounding name (as compared to a White-sounding name). But we can actually use our regression framework to look for evidence of *heterogeneous treatment effects*—effects that are different for different types of people in our data. We accomplish this by *interacting* a variable we think may be related to experiencing a differential treatment effect with our treatment variable. For example, if we think that applicants with Black-sounding names who have a college degree are likely to experience less discrimination, we can interact `black` with an indicator for having a college degree. If having a college degree reduces discrimination, we could expect the interaction term to be positive. 

Is there more or less racial discrimination (the absolute magnitude difference in call back rates between Black and White applicants) among applicants who have a college degree? Store your answer as the string `"more discrimination"` or `"less discrimination"` under the key `"ex8_college_heterogeneity"`.

Please still include `education`, `yearsexp`, `female`, and `computerskills` as controls.

**Note:** it's relatively safe to assume that someone hiring employees who sees a resume that does *not* report education levels will assume the applicant does not have a college degree. So treat "No education reported" as "not having a college degree."

In percentage points, what is the difference in call back rates:

- between White applicants without a college degree and Black applicants without a college degree (`ex8_black_nocollege`).
- between White applicants with a college degree and Black applicants with a college degree (`ex8_black_college`).

Use negative values to denote a lower probability for Black applicants to get a call back. **Scale so a value of `1` is a one percentage point difference. Please round your answer to 2 percentage points.**

Focus on the coefficient values, even if the significance is low.

In [ ]:
# Treat "No education reported" (education == 0) as no college degree.
# Here, assume education = 4 means applicant has a college degree.
df["college_degree"] = (df["education"] == 4).astype(int)

ex8_model = smf.ols(
    "call ~ black * college_degree + education + yearsexp + female + computerskills",
    data=df,
).fit(cov_type="HC3")

black_nocollege_pp = ex8_model.params["black"] * 100

interaction_term = (
    "black:college_degree"
    if "black:college_degree" in ex8_model.params
    else "college_degree:black"
)
black_college_pp = (
    ex8_model.params["black"] + ex8_model.params[interaction_term]
) * 100

results["ex8_college_heterogeneity"] = (
    "more discrimination"
    if abs(black_college_pp) > abs(black_nocollege_pp)
    else "less discrimination"
)
results["ex8_black_nocollege"] = round(black_nocollege_pp, 2)
results["ex8_black_college"] = round(black_college_pp, 2)

results["ex8_black_nocollege"], results["ex8_black_college"], results[
    "ex8_college_heterogeneity"
]

(np.float64(-4.05), np.float64(-2.82), 'less discrimination')

### Exercise 9

Now let's compare men and women—is the penalty for having a Black-sounding name greater for Black men or Black women? Store your answer as `"greater discrimination for men"` or `"greater discrimination for women"` in `"ex9_gender_and_discrimination"`.

Focus on the coefficient values, even if the significance is low.

Again, please still include `education`, `yearsexp`, `female`, and `computerskills` as controls.

In [ ]:
ex9_model = smf.ols(
    "call ~ black * female + education + yearsexp + female + computerskills", data=df
).fit(cov_type="HC3")

black_men_pp = ex9_model.params["black"] * 100

interaction_term = (
    "black:female" if "black:female" in ex9_model.params else "female:black"
)
black_women_pp = (ex9_model.params["black"] + ex9_model.params[interaction_term]) * 100

results["ex9_gender_and_discrimination"] = (
    "greater discrimination for men"
    if abs(black_men_pp) > abs(black_women_pp)
    else "greater discrimination for women"
)

results["black_men_pp"] = round(black_men_pp, 2)
results["black_women_pp"] = round(black_women_pp, 2)


results["black_men_pp"], results["black_women_pp"], results[
    "ex9_gender_and_discrimination"
]

(np.float64(-2.87), np.float64(-3.25), 'greater discrimination for women')

### Exercise 10

Calculate and/or lookup the following online:

- What is the share of applicants in our dataset with college degrees?
- What share of Black adult Americans have college degrees (i.e. have completed a bachelors degree)?

Is the share of Black applicants with college degrees in this data `"greater"`, or `"less"` than in the US? Store your answer as one of those strings in `"ex10_experiment_v_us"`

In [ ]:
# Share of all applicants with college degrees in dataset
share_college_all = df["college_degree"].mean()

# Share of black applicants with college degrees in dataset
share_college_black_experiment = df.loc[df["black"] == 1, "college_degree"].mean()

# Share of Black adult Americans with college degrees (from google)
share_college_black_us = 0.27

results["ex10_experiment_v_us"] = (
    "greater" if share_college_black_experiment > share_college_black_us else "less"
)

round(share_college_all, 2), round(
    share_college_black_experiment, 2
), share_college_black_us, results["ex10_experiment_v_us"]

(np.float64(0.72), np.float64(0.72), 0.27, 'greater')

### Exercise 11

Bearing in mind your answers to Exercise 8 and to Exercise 10, how do you think the Average Treatment Effect you estimated in Exercises 5 and 6 might generalize to the experience of the average Black American (i.e., how do you think the ATE for the average Black American would compare to the ATE estimated from this experiment)?


The Average Treatment Effect (ATE) of -3.20% estimated in Exercises 5 and 6 likely understates the magnitude of discrimination experienced by the average Black American.

According to Exercise 8, there is significant heterogeneity in treatment effects: applicants without a college degree face a much larger penalty (-4.05 percentage points) than those with a degree (-2.82 percentage points). However, Exercise 10 shows that the experimental sample is not representative of the US population; it is heavily composed of college graduates (93%), whereas only about 27% of Black adult Americans hold a degree.

Since the average Black American is far more likely to belong to the "no college degree" group, which experiences more severe discrimination, the true ATE in the broader US population would likely be more negative (larger in absolute magnitude) than the -3.20% estimated from this high-skilled experimental sample.

### Exercise 12

What does your answer to Exercise 10 imply about the study's *internal* validity?

From Exercise 10, the internal validity doesn't seem violated. The treatment (Black- vs. White-sounding names) was randomly assigned, so differences in education levels between the experimental sample and the U.S. population do not affect the credibility of the causal estimate within the experiment.

### Exercise 13

What does your answer to Exercise 10 imply about the study's *external* validity?

From Exercise 10, the external validity might be violated. Because the experimental sample contains a much higher share of college-educated Black applicants than the general Black population, the estimated ATE may not generalize well to the average Black American and likely understates the magnitude of discrimination in the broader population

## What Did We Just Measure?

It's worth pausing for a moment to think about exactly what we've measured in this experiment. Was it the effect of race on hiring? Or the difference in the experience of the average White job applicant from the average Black job applicant?

Well... no. What we have measured in this experiment is **just** the effect of having a Black-sounding name (as opposed to a White-sounding name) on your resume on the likelihood of getting a followup call from someone hiring in Boston or Chicago given identical resumes. In that sense, what we've measured is a small *piece* of the difference in the experience of Black and White Americans when seeking employment. As anyone looking for a job knows, getting a call-back is obviously a crucial step in getting a job, so this difference—even if it's just one part of the overall difference—is remarkable.

In [18]:
from pprint import pprint

results.pop("black_men_pp", None)
results.pop("black_women_pp", None)

pprint(results)

{'ex10_experiment_v_us': 'greater',
 'ex2_pvalue_computerskills': np.float64(0.03),
 'ex2_pvalue_female': np.float32(0.38),
 'ex2_pvalue_yearsexp': np.float64(0.85),
 'ex3_pvalue_education': np.float64(0.49),
 'ex4_validity': 'internal',
 'ex5_pvalue': np.float32(4e-05),
 'ex5_white_advantage_percent': np.float32(49.68),
 'ex5_white_advantage_percentage_points': np.float32(3.2),
 'ex6_black_pvalue': np.float64(4e-05),
 'ex8_black_college': np.float64(-2.82),
 'ex8_black_nocollege': np.float64(-4.05),
 'ex8_college_heterogeneity': 'less discrimination',
 'ex9_gender_and_discrimination': 'greater discrimination for women'}


In [19]:
assert set(results.keys()) == {
    "ex2_pvalue_computerskills",
    "ex2_pvalue_female",
    "ex2_pvalue_yearsexp",
    "ex3_pvalue_education",
    "ex4_validity",
    "ex5_pvalue",
    "ex5_white_advantage_percent",
    "ex5_white_advantage_percentage_points",
    "ex6_black_pvalue",
    "ex8_black_college",
    "ex8_black_nocollege",
    "ex8_college_heterogeneity",
    "ex9_gender_and_discrimination",
    "ex10_experiment_v_us",
}